In [1]:
#Initial environement of computations

import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests


number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

#Access and recess equation form the Parisian Alfonsines Tables
Hist_table=HTable.read(309,format='dishas')
Hist_table

Residual,Tables [60]
deg,
IntegerAndSexagesimal,void128
1 ;,"(1 ; , 0 ; 10) .. (60 ; , 10 ; 00)"
2 ;,"(1 ; , 0 ; 20) .. (60 ; , 20 ; 00)"
3 ;,"(1 ; , 0 ; 30) .. (60 ; , 30 ; 00)"
4 ;,"(1 ; , 0 ; 40) .. (60 ; , 40 ; 00)"
5 ;,"(1 ; , 0 ; 50) .. (60 ; , 50 ; 00)"
6 ;,"(1 ; , 1 ; 00) .. (60 ; , 60 ; 00)"
7 ;,"(1 ; , 1 ; 10) .. (60 ; , 70 ; 00)"
8 ;,"(1 ; , 1 ; 20) .. (60 ; , 80 ; 00)"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(8):
    for j in range(60):
        Test_recomp[i][1][j][1]=Sexagesimal('0;')
        Delta_recomp[i][1][j][1]=Sexagesimal('0;')


In [4]:
for i in range(8):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
        # initial value for dividend
        dividend1=Hist_table[i][0]
                 
    for j in range(60):
        with set_precision(tmode=TruncatureMode.TRUNC, pmode=2):
            # second values for dividend
            dividend2=Hist_table[0][1][j][0]
         
        # computation of the proportion
        with set_precision(tmode=TruncatureMode.ROUND, pmode=1):
            prop=dividend1*dividend2/6
            Test_recomp[i][1][j][1]=prop*u.degree
            hist=Hist_table[i][1][j][1]
            Delta_recomp[i][1][j][1]=60*60*(hist-prop)

In [5]:
# residuals visualisation and statistical summaries 1
A=Delta_recomp.to_pandas()
(A.mean().mean(), A.std().std())

(0.0, 0.0)

In [6]:
# residuals visualisation and statistical summaries 2
A.astype(float).style.background_gradient(axis=None)

,1 ;,2 ;,3 ;,4 ;,5 ;,6 ;,7 ;,8 ;
1 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10 ;,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Prop 6°_TM.csv')
B.to_csv('Prop 6°_TM.zip', index=False, compression=compression_opts)